# Importamos las librerías necesarias

In [52]:
import pandas as pd
import numpy
import matplotlib
from datetime import datetime
import pprint

# Leemos, extraemos datos y hacemos dos copias


In [53]:
data = pd.read_csv('ufo_sighting_data.csv')
df = pd.DataFrame(data)
ufo = data.copy()
ufo2 = data.copy()

# Comprobamos valores nulos

In [11]:
ufo.isnull().sum()

Date_time                             0
city                                  0
state/province                     5797
country                            9670
UFO_shape                          1932
length_of_encounter_seconds           0
described_duration_of_encounter       0
description                          15
date_documented                       0
latitude                              0
longitude                             0
dtype: int64

# Limpiamos la segunda copia de datos (ufo2)

In [54]:
ufo2 = ufo2.drop(ufo2[(ufo2['state/province'].isnull()) |
                      (ufo2['country'].isnull()) | 
                      (ufo2['UFO_shape'].isnull()) | 
                      (ufo2['description'].isnull()) ].index)

# Nuestro estudio se va a centrar por una parte en las latitudes y por otra parte en los meses y en los años


## Tratamiento de las fechas

## Funciones procedentes del refactor del análisis y que utilizaremos

In [55]:
def applyToInt(df, col):
    return [int(item) for item in df[col]]

def extractByIndexAndSeparator(df, col, separator, index):
    return [item.split(separator)[index] for item in df[col]]

def splitDataBySeparator(df, col, separator):
    return [item.split(separator) for item in df[col]]

def extractItemByIndex(df, col, index):
    return [item[index] for item in df[col]]

def getFrequencies(df, col):
    dictionary = {}
    sorted_df_col = sorted(df[col])
    unique_items = set(sorted_df_col)
    for item in unique_items:
        dictionary[item] = sorted_df_col.count(item)
    
    return dictionary

### Obtenemos las fechas y las horas a partir de Date_time

In [56]:
ufo['Date'] = extractByIndexAndSeparator(ufo, 'Date_time', ' ', 0)
ufo['time'] = extractByIndexAndSeparator(ufo, 'Date_time' , ' ', 1)
ufo['Date'] = ufo['Date'].str.strip()

ufo2['Date'] = extractByIndexAndSeparator(ufo2, 'Date_time', ' ', 0)
ufo2['time'] = extractByIndexAndSeparator(ufo2, 'Date_time' , ' ', 1)

### Obtenemos los meses y los años a partir de Date

In [89]:
ufo['months_aux'] = splitDataBySeparator(ufo,'Date', '/')
ufo['months'] = extractItemByIndex(ufo, 'months_aux', 0 )
ufo['years'] = extractItemByIndex(ufo, 'months_aux', 2 )
ufo['years'] = applyToInt(ufo, 'years')
annual_freq = getFrequencies(ufo, 'years')

ufo2['months_aux'] = splitDataBySeparator(ufo2,'Date', '/')
ufo2['months']= extractItemByIndex(ufo2, 'months_aux', 0 )
ufo2['years'] = extractItemByIndex(ufo2, 'months_aux', 2 )
ufo2['years'] = applyToInt(ufo2, 'years')
annual_freq2 = getFrequencies(ufo2, 'years')



## Tratamiento de las latitudes

In [ ]:
ufo2['latitude'].nunique() == ufo['latitude'].nunique()
ufo2['latitude'].nunique()
ufo_removed_latitudes = [item for item in ufo['latitude'] if item not in ufo2['latitude']]
#clean_latitudes = [item.str.replace(('\w*'), '') for item in ufo['latitude']]
#ufo_removed_latitudes_to_int = [float(item) for item in ufo_removed_latitudes]
len(ufo_removed_latitudes)
ufo['latitude'].nunique()
removed_latitudes_length = len(ufo_removed_latitudes)
removed_latitudes_unique_length = len(set(ufo_removed_latitudes))
removed_latitudes_unique_length
ufo_removed_unique_latitudes = set(ufo_removed_latitudes)
ufo_removed_unique_latitudes